In [2]:
import os

In [2]:
files = os.listdir('temp')

In [3]:
files

['temp01.txt', 'temp02.txt', 'temp03.txt']

In [4]:
type(files[0])

str

In [11]:
x = 'sdfsd.txt'
file_extension = x.split('.')[-1]
print(file_extension)

txt


creating dumb files 

In [47]:
path = 'temp/'
for i in range(10):
    file_name = f"temp_{i:02}.txt"
    file_path = os.path.join(path, file_name)
    with open(file_path, mode='x') as file:
        pass

removing directory contents

In [46]:
files = os.listdir('dataset')
src_path = 'dataset'

for f in files:
    file_path = f'{src_path}/{f}'
    os.remove(file_path)


copying directory contents to another directory

In [38]:
import shutil

In [48]:
files = os.listdir('temp')
dst_dir = 'dataset'
for f in files:
    file_to_copy = f"temp/{f}"
    shutil.copy2(file_to_copy, dst_dir)

In [64]:
def split(dir_path: str, train_s: int, valid_s: int, test_s: int):
    check_size = train_s + valid_s + test_s
    if check_size != 100:
        raise Exception('make sure that your split data size is correct!')

    curr_path = f'{os.getcwd()}/dataset'
    os.mkdir(curr_path)
    files = os.listdir(dir_path)

    #number of elements
    num_elements = get_elements(files, train_s, valid_s, test_s)

    #creating train dir
    train_path = f'{curr_path}/train'
    os.mkdir(train_path)
    create_train(files, dir_path, train_path, num_elements['train'])

    #creating valid dir
    valid_path = f'{curr_path}/valid'
    os.mkdir(valid_path)
    valid_index = num_elements['train']
    create_valid(files, dir_path, valid_path, num_elements['valid'], valid_index)

    #creating temp dir
    test_path = f'{curr_path}/test'
    os.mkdir(test_path)
    test_index = num_elements['train'] + num_elements['valid']
    create_test(files, dir_path, test_path, num_elements['temp'], test_index)


def get_elements(f_list, train_s: int, valid_s: int, test_s: int):
    train_e = len(f_list) * train_s / 100
    if train_e % 1 != 0:
        train_e = int(train_e) + 1
        valid_e = len(f_list) * valid_s // 100
        test_e = len(f_list) * test_s // 100
    else:
        valid_e = len(f_list) * valid_s / 100
        if valid_e % 1 != 0:
            valid_e = int(valid_e) + 1
            test_e = len(f_list) * test_s // 100
        else:
            test_e = len(f_list) * test_s / 100
            if test_e % 1 != 0:
                test_e = int(test_e) + 1

    elements = {
        'train': int(train_e),
        'valid': int(valid_e),
        'temp': int(test_e)
    }
    return elements


def create_train(files_list, src_path, dst_path, num_elements):
    for i in range(num_elements):
        file_to_copy = f"{src_path}/{files_list[i]}"
        shutil.copy2(file_to_copy, dst_path)


def create_valid(files_list, src_path, dst_path, num_elements, start_index):
    j = start_index
    for i in range(num_elements):
        file_to_copy = f"{src_path}/{files_list[j]}"
        shutil.copy2(file_to_copy, dst_path)
        j += 1


def create_test(files_list, src_path, dst_path, num_elements, start_index):
    print(f"inside valid -> start_index = {start_index}, num_elements = {num_elements}")
    j = start_index
    for i in range(num_elements):
        file_to_copy = f"{src_path}/{files_list[j]}"
        shutil.copy2(file_to_copy, dst_path)
        j += 1